In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import numpy as np
import random

In [2]:
#useful variables
DATA_DIR = '../../data/'
IMG_DIR = '../../imgs/'
worlds = ['overworld', 'nether', 'end']
BUFFER = 500
#each row, minX, maxX, minZ, maxZ
world_bounds = {worlds[0]:[0,0,0,0], #overworld
                worlds[1]:[0,0,0,0], #nether
                worlds[2]:[0,0,0,0]} #end

TICKS_PER_24HR = 1728000

### Get Unique Players and Colors

In [3]:
player_df = pd.read_csv(DATA_DIR + 'overworld/PlayerMoveEvent.csv')
players = player_df['player'].unique()
num_players = len(players)

player_mapping = {players[i]:i for i in range(len(players))}

### Generate random colors

In [4]:
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
colors = get_colors(num_players)

### Find world bounds for each world

In [5]:
max_time = 0

# gets the bounds of the data for each world
for i, world in enumerate(worlds):
    ref_csv = pd.read_csv(DATA_DIR + world + '/PlayerMoveEvent.csv')
    x_coords = ref_csv['location.x']
    z_coords = ref_csv['location.z']
    max_time = max(max_time, max(ref_csv['worldTime']))
    
    world_bounds[world][0] = min(x_coords) - BUFFER
    world_bounds[world][1] = max(x_coords) + BUFFER
    world_bounds[world][2] = min(z_coords) - BUFFER
    world_bounds[world][3] = max(z_coords) + BUFFER

### Create Folders

In [6]:
num_days = math.ceil(max_time / TICKS_PER_24HR)

# folers for each separate day
for d in range(num_days):
    for world in worlds:
        for p in range(len(players)):
            os.makedirs(IMG_DIR + 'day_'+str(d) + '/' + world + '/' + 'player_'+str(p)+'/', exist_ok=True)

# folders for ALL days
for world in worlds:
    for p in range(len(players)):
        os.makedirs(IMG_DIR + 'ALL' + '/' + world + '/' + 'player_'+str(p)+'/', exist_ok=True)

### Generate images for each day

In [7]:
# for each world
for i, world in enumerate(worlds):
    
    # get all files in world directory
    curr_dir = DATA_DIR + world + '/'
    all_files = [f for f in os.listdir(curr_dir) if os.path.isfile(os.path.join(curr_dir, f))]
    
    #for each event
    for f in all_files:
        event_name = f[:-4]
        temp_df = pd.read_csv(curr_dir + f)
        temp_df['worldTime'] = np.floor(temp_df['worldTime'] / TICKS_PER_24HR)
        
        temp_unique_days = list(temp_df['worldTime'].unique())
        
        # for each day
        for day in temp_unique_days:
            temp_time_df = temp_df[temp_df['worldTime'] == day]
            unique_players_at_time = list(temp_time_df['player'].unique())
            
            # for each player
            for player in unique_players_at_time:
                temp_player_df = temp_time_df[temp_time_df['player'] == player]
                
                #Get data
                x_data = temp_player_df['location.x']
                z_data = temp_player_df['location.z']
                player_idx = player_mapping[player]
                #set bounds of plot
                plt.xlim([world_bounds[world][0], world_bounds[world][1]])
                plt.ylim([world_bounds[world][2], world_bounds[world][3]])
                
                #makt the plot
                # plt.title(world+ '_day'+str(int(day))+'_player'+str(player_idx)+'_'+event_name)
                plt.scatter(x_data,z_data, c=colors[player_idx], s = 1)
                
                filename = event_name + '.png'
                filepath = IMG_DIR + 'day_' + str(int(day)) + '/' + world + '/player_' + str(player_idx) + '/' + filename
                plt.savefig(filepath, format='png', dpi=150, transparent=True)
                plt.clf()
        

<Figure size 432x288 with 0 Axes>

### Generate images across all days

In [8]:
# for each world
for i, world in enumerate(worlds):
    
    # get all files in world directory
    curr_dir = DATA_DIR + world + '/'
    all_files = [f for f in os.listdir(curr_dir) if os.path.isfile(os.path.join(curr_dir, f))]
    
    #for each event
    for f in all_files:
        event_name = f[:-4]
        temp_df = pd.read_csv(curr_dir + f)
        temp_df['worldTime'] = np.floor(temp_df['worldTime'] / TICKS_PER_24HR)
        
        unique_players_at_time = list(temp_df['player'].unique())
            
        #for each player
        for player in unique_players_at_time:
            temp_player_df = temp_df[temp_df['player'] == player]

            #Get data
            x_data = temp_player_df['location.x']
            z_data = temp_player_df['location.z']
            player_idx = player_mapping[player]
            #set bounds of plot
            plt.xlim([world_bounds[world][0], world_bounds[world][1]])
            plt.ylim([world_bounds[world][2], world_bounds[world][3]])

            #makt the plot
            # plt.title(world+ '_ALL-DAYS'+'_player'+str(player_idx)+'_'+event_name)
            plt.scatter(x_data,z_data, c=colors[player_idx], s = 1)

            filename = event_name + '.png'
            filepath = IMG_DIR + 'ALL'+ '/' + world + '/player_' + str(player_idx) + '/' + filename
            plt.savefig(filepath, format='png', dpi=150, transparent=True)
            plt.clf()
        

<Figure size 432x288 with 0 Axes>